# **Transfer Learning**

**Goal**: Using punlicly available image classification model and alter it to meet our needs, vastly improving both training speed and model performance.

**Objectives**

- Load data.
- Download a pre-trained model.
- Alter that model to suit our needs with Transfer Learning.
- Train just the altered part.
- Test model performance with K-fold cross-validation.

In [1]:
import os 

import matplotlib